In [1]:
# ============================================================================
# FIX DEFINITIVO - CORRECCIÓN DE DIMENSIONES DE OBSERVACIÓN
# Solución al error: could not broadcast input array from shape (7,) into shape (10,)
# ============================================================================

import pybullet as p
import pybullet_data
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import time
import math
import random
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

print("🔧 BRAZO ROBÓTICO - VERSIÓN CON DIMENSIONES CORREGIDAS")
print("=" * 60)

🔧 BRAZO ROBÓTICO - VERSIÓN CON DIMENSIONES CORREGIDAS


In [2]:
# ============================================================================
# ENTORNO CORREGIDO CON DIMENSIONES CONSISTENTES
# ============================================================================

class FixedRobotArmEnv(gym.Env):
    """Entorno del brazo robótico con dimensiones de observación corregidas"""
    
    def __init__(self, render_mode='human'):
        super(FixedRobotArmEnv, self).__init__()
        
        self.render_mode = render_mode
        self.max_steps = 200
        self.current_step = 0
        
        # Variables importantes
        self.robot_id = None
        self.cup_id = None
        self.base_id = None
        self.num_joints = 0
        
        # Conectar PyBullet
        self._connect_pybullet()
        self._setup_world()
        self._create_robot()
        self._create_cup()
        
        # DIMENSIONES FIJAS Y CONSISTENTES
        # Definir exactamente 6 articulaciones para consistencia
        self.n_joints = 6
        
        # Espacio de acciones: 6 articulaciones
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=(6,), dtype=np.float32
        )
        
        # Espacio de observaciones: EXACTAMENTE 13 elementos
        # [6 pos articulaciones] + [3 pos vaso] + [3 pos end-effector] + [1 distancia] = 13
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(13,), dtype=np.float32
        )
        
        print(f"✅ Entorno creado - Observaciones: {self.observation_space.shape}")
        print(f"✅ Acciones: {self.action_space.shape}")
    
    def _connect_pybullet(self):
        """Conectar a PyBullet de forma segura"""
        try:
            p.disconnect()
        except:
            pass
        
        if self.render_mode == 'human':
            self.physics_client = p.connect(p.GUI)
        else:
            self.physics_client = p.connect(p.DIRECT)
    
    def _setup_world(self):
        """Configurar mundo de simulación"""
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)
        
        if self.render_mode == 'human':
            p.configureDebugVisualizer(p.COV_ENABLE_GUI, 1)
            p.resetDebugVisualizerCamera(
                cameraDistance=1.8,
                cameraYaw=45,
                cameraPitch=-30,
                cameraTargetPosition=[0, 0, 0.4]
            )
        
        # Cargar plano
        self.plane_id = p.loadURDF("plane.urdf")
    
    def _create_robot(self):
        """Crear brazo robótico con exactamente 6 articulaciones"""
        # Base
        base_collision = p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.05])
        base_visual = p.createVisualShape(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.05], 
                                         rgbaColor=[0.3, 0.3, 0.3, 1])
        
        self.base_id = p.createMultiBody(
            baseMass=20,
            baseCollisionShapeIndex=base_collision,
            baseVisualShapeIndex=base_visual,
            basePosition=[0, 0, 0.05]
        )
        
        # Intentar cargar KUKA
        try:
            self.robot_id = p.loadURDF("kuka_iiwa/model.urdf", [0, 0, 0.1], useFixedBase=True)
            if self.robot_id >= 0:
                self.num_joints = p.getNumJoints(self.robot_id)
                print(f"🤖 KUKA cargado con {self.num_joints} articulaciones")
            else:
                raise Exception("KUKA no disponible")
        except:
            print("🔧 Creando brazo manual...")
            self._create_manual_robot()
        
        print(f"🔗 Robot configurado con {self.num_joints} articulaciones")
    
    def _create_manual_robot(self):
        """Crear brazo manual con EXACTAMENTE 6 articulaciones"""
        
        # Colores para cada eslabón
        colors = [
            [0.8, 0.2, 0.2, 1],  # Rojo
            [0.2, 0.8, 0.2, 1],  # Verde
            [0.2, 0.2, 0.8, 1],  # Azul
            [0.8, 0.8, 0.2, 1],  # Amarillo
            [0.8, 0.2, 0.8, 1],  # Magenta
            [0.2, 0.8, 0.8, 1],  # Cian
        ]
        
        # Especificaciones de 6 eslabones
        link_specs = [
            (0.04, 0.12, 1.5),  # radio, altura, masa
            (0.035, 0.20, 1.2),
            (0.03, 0.18, 1.0),
            (0.025, 0.15, 0.8),
            (0.02, 0.10, 0.6),
            (0.025, 0.08, 0.5),
        ]
        
        # Ejes de rotación
        joint_axes = [
            [0, 0, 1],  # Base Z
            [0, 1, 0],  # Hombro Y
            [0, 1, 0],  # Codo Y
            [1, 0, 0],  # Antebrazo X
            [0, 0, 1],  # Muñeca Z
            [1, 0, 0],  # Gripper X
        ]
        
        # Construir listas para createMultiBody
        link_masses = []
        link_collision_shapes = []
        link_visual_shapes = []
        link_positions = []
        link_orientations = []
        link_inertial_frame_positions = []
        link_inertial_frame_orientations = []
        link_parent_indices = []
        link_joint_types = []
        link_joint_axis = []
        
        for i, ((radius, height, mass), color, axis) in enumerate(zip(link_specs, colors, joint_axes)):
            # Crear formas
            collision_shape = p.createCollisionShape(p.GEOM_CYLINDER, radius=radius, height=height)
            visual_shape = p.createVisualShape(p.GEOM_CYLINDER, radius=radius, length=height, rgbaColor=color)
            
            # Agregar a listas
            link_masses.append(mass)
            link_collision_shapes.append(collision_shape)
            link_visual_shapes.append(visual_shape)
            link_positions.append([0, 0, height/2])
            link_orientations.append([0, 0, 0, 1])
            link_inertial_frame_positions.append([0, 0, height/2])
            link_inertial_frame_orientations.append([0, 0, 0, 1])
            link_parent_indices.append(i)
            link_joint_types.append(p.JOINT_REVOLUTE)
            link_joint_axis.append(axis)
        
        # Crear robot
        self.robot_id = p.createMultiBody(
            baseMass=0,
            basePosition=[0, 0, 0.15],
            linkMasses=link_masses,
            linkCollisionShapeIndices=link_collision_shapes,
            linkVisualShapeIndices=link_visual_shapes,
            linkPositions=link_positions,
            linkOrientations=link_orientations,
            linkInertialFramePositions=link_inertial_frame_positions,
            linkInertialFrameOrientations=link_inertial_frame_orientations,
            linkParentIndices=link_parent_indices,
            linkJointTypes=link_joint_types,
            linkJointAxis=link_joint_axis
        )
        
        self.num_joints = 6  # Forzar exactamente 6 articulaciones
        print("✅ Brazo manual con 6 articulaciones creado")
    
    def _create_cup(self):
        """Crear vaso objetivo"""
        x = random.uniform(-0.4, 0.4)
        y = random.uniform(-0.4, 0.4)
        z = 0.06
        
        cup_collision = p.createCollisionShape(p.GEOM_CYLINDER, radius=0.03, height=0.10)
        cup_visual = p.createVisualShape(p.GEOM_CYLINDER, radius=0.03, length=0.10,
                                        rgbaColor=[0.1, 0.3, 0.9, 0.8])
        
        self.cup_id = p.createMultiBody(
            baseMass=0.15,
            baseCollisionShapeIndex=cup_collision,
            baseVisualShapeIndex=cup_visual,
            basePosition=[x, y, z]
        )
        
        self.cup_initial_pos = np.array([x, y, z])
        print(f"🥤 Vaso creado en: ({x:.2f}, {y:.2f}, {z:.2f})")
    
    def _get_observation(self):
        """Obtener observación con DIMENSIONES EXACTAS"""
        
        # 1. Posiciones de articulaciones (EXACTAMENTE 6 valores)
        joint_positions = np.zeros(6, dtype=np.float32)
        
        if self.robot_id is not None:
            actual_joints = min(self.num_joints, 6)
            for i in range(actual_joints):
                try:
                    joint_state = p.getJointState(self.robot_id, i)
                    joint_positions[i] = float(joint_state[0])
                except:
                    joint_positions[i] = 0.0
        
        # 2. Posición del vaso (EXACTAMENTE 3 valores)
        if self.cup_id is not None:
            try:
                cup_pos, _ = p.getBasePositionAndOrientation(self.cup_id)
                cup_position = np.array(cup_pos, dtype=np.float32)
            except:
                cup_position = np.zeros(3, dtype=np.float32)
        else:
            cup_position = np.zeros(3, dtype=np.float32)
        
        # 3. Posición del end-effector (EXACTAMENTE 3 valores)
        if self.robot_id is not None and self.num_joints > 0:
            try:
                end_effector_state = p.getLinkState(self.robot_id, self.num_joints - 1)
                end_effector_pos = np.array(end_effector_state[0], dtype=np.float32)
            except:
                end_effector_pos = np.zeros(3, dtype=np.float32)
        else:
            end_effector_pos = np.zeros(3, dtype=np.float32)
        
        # 4. Distancia al objetivo (EXACTAMENTE 1 valor)
        distance = np.linalg.norm(end_effector_pos - cup_position)
        distance = np.array([distance], dtype=np.float32)
        
        # Combinar observación: 6 + 3 + 3 + 1 = 13 elementos EXACTOS
        observation = np.concatenate([
            joint_positions,    # 6 elementos
            cup_position,       # 3 elementos
            end_effector_pos,   # 3 elementos
            distance           # 1 elemento
        ])
        
        # VERIFICACIÓN CRÍTICA de dimensiones
        assert observation.shape == (13,), f"Error: observación tiene forma {observation.shape}, esperada (13,)"
        
        return observation.astype(np.float32)
    
    def reset(self, seed=None, options=None):
        """Reiniciar entorno"""
        super().reset(seed=seed)
        
        self.current_step = 0
        
        # Remover vaso anterior
        if self.cup_id is not None:
            try:
                p.removeBody(self.cup_id)
            except:
                pass
        
        # Crear nuevo vaso
        self._create_cup()
        
        # Reiniciar robot a posición inicial
        if self.robot_id is not None:
            home_positions = [0, 0, -0.3, 0, 0, 0]
            actual_joints = min(self.num_joints, 6)
            for i in range(actual_joints):
                try:
                    p.resetJointState(self.robot_id, i, home_positions[i])
                except:
                    pass
        
        # Estabilizar simulación
        for _ in range(30):
            p.stepSimulation()
            if self.render_mode == 'human':
                time.sleep(1./240.)
        
        # Obtener observación inicial
        obs = self._get_observation()
        
        # VERIFICACIÓN FINAL
        assert obs.shape == (13,), f"Reset: observación incorrecta {obs.shape}"
        
        info = {}
        return obs, info
    
    def step(self, action):
        """Ejecutar una acción"""
        self.current_step += 1
        
        # VERIFICAR dimensión de acción
        action = np.array(action, dtype=np.float32)
        if action.shape != (6,):
            # Ajustar acción si es necesario
            if len(action) > 6:
                action = action[:6]
            else:
                action = np.pad(action, (0, 6 - len(action)), 'constant')
        
        # Aplicar acciones a las articulaciones
        if self.robot_id is not None:
            actual_joints = min(self.num_joints, 6)
            for i in range(actual_joints):
                try:
                    target_angle = action[i] * math.pi
                    p.setJointMotorControl2(
                        self.robot_id, i, p.POSITION_CONTROL,
                        targetPosition=target_angle, force=150
                    )
                except:
                    pass
        
        # Simular física
        p.stepSimulation()
        if self.render_mode == 'human':
            time.sleep(1./240.)
        
        # Obtener nueva observación
        obs = self._get_observation()
        
        # VERIFICACIÓN CRÍTICA
        assert obs.shape == (13,), f"Step: observación incorrecta {obs.shape}"
        
        # Calcular recompensa
        reward = self._calculate_reward(obs)
        
        # Verificar terminación
        terminated = self._is_terminated(obs)
        truncated = self.current_step >= self.max_steps
        
        info = {
            'distance': float(obs[-1]),
            'success': float(obs[-1]) < 0.1,
            'step': self.current_step
        }
        
        return obs, reward, terminated, truncated, info
    
    def _calculate_reward(self, obs):
        """Calcular recompensa basada en observación"""
        distance = float(obs[-1])  # Último elemento es la distancia
        
        # Recompensa base: inversa de la distancia
        reward = -distance * 3
        
        # Bonificaciones por proximidad
        if distance < 0.05:
            reward += 100  # Muy cerca
        elif distance < 0.1:
            reward += 50   # Cerca
        elif distance < 0.2:
            reward += 25   # Acercándose
        elif distance < 0.3:
            reward += 10   # En camino
        
        # Verificar manipulación del vaso
        cup_pos = obs[6:9]  # Posición del vaso en observación
        displacement = np.linalg.norm(cup_pos - self.cup_initial_pos)
        
        if displacement > 0.03:
            reward += 30  # Bonus por mover el vaso
        
        # Penalización leve por tiempo
        reward -= 0.2
        
        return float(reward)
    
    def _is_terminated(self, obs):
        """Verificar si el episodio terminó"""
        distance = float(obs[-1])
        
        # Éxito: muy cerca del vaso
        if distance < 0.05:
            return True
        
        # Éxito: vaso movido significativamente
        cup_pos = obs[6:9]
        displacement = np.linalg.norm(cup_pos - self.cup_initial_pos)
        if displacement > 0.08:
            return True
        
        return False
    
    def render(self):
        """Renderizar entorno"""
        pass
    
    def close(self):
        """Cerrar entorno"""
        try:
            p.disconnect()
        except:
            pass


In [3]:
# ============================================================================
# FUNCIONES DE PRUEBA Y ENTRENAMIENTO
# ============================================================================

def test_environment_dimensions():
    """Probar entorno y verificar dimensiones"""
    print("\n🧪 PROBANDO DIMENSIONES DEL ENTORNO")
    print("=" * 40)
    
    env = FixedRobotArmEnv(render_mode='human')
    
    print(f"📊 Espacio de observaciones: {env.observation_space.shape}")
    print(f"🎮 Espacio de acciones: {env.action_space.shape}")
    
    # Verificar con stable-baselines3
    try:
        check_env(env)
        print("✅ Entorno compatible con Stable-Baselines3")
    except Exception as e:
        print(f"⚠️ Advertencia: {e}")
    
    # Probar reset y step
    print("\n🔄 Probando reset...")
    obs, info = env.reset()
    print(f"✅ Reset OK - Observación: {obs.shape}")
    
    print("\n▶️ Probando steps...")
    for i in range(50):
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        
        if i % 10 == 0:
            print(f"   Paso {i}: obs={obs.shape}, distancia={info['distance']:.3f}")
        
        if terminated or truncated:
            print(f"   Episodio terminó en paso {i}")
            obs, info = env.reset()
            print(f"   Reset OK - Nueva observación: {obs.shape}")
    
    env.close()
    print("✅ Prueba de dimensiones completada exitosamente")

def train_fixed_robot():
    """Entrenar robot con dimensiones corregidas"""
    print("\n🚀 ENTRENAMIENTO CON DIMENSIONES CORREGIDAS")
    print("=" * 50)
    
    # Crear entorno de entrenamiento
    train_env = FixedRobotArmEnv(render_mode='direct')
    
    # Verificar entorno
    try:
        check_env(train_env)
        print("✅ Entorno verificado correctamente")
    except Exception as e:
        print(f"❌ Error en verificación: {e}")
        train_env.close()
        return
    
    # Configurar PPO
    print("🧠 Configurando PPO...")
    model = PPO(
        "MlpPolicy",
        train_env,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        verbose=1
    )
    
    print("✅ PPO configurado correctamente")
    
    # Entrenar
    print("📚 Iniciando entrenamiento...")
    
    # Entrenamiento en fases
    phases = [
        (15000, "🌱 Exploración inicial"),
        (20000, "🎯 Refinamiento"),
        (25000, "🏆 Optimización"),
        (20000, "✨ Perfeccionamiento")
    ]
    
    for timesteps, phase_name in phases:
        print(f"\n{phase_name} - {timesteps} pasos")
        model.learn(total_timesteps=timesteps, reset_num_timesteps=False)
        
        # Evaluación rápida
        print("📊 Evaluación...")
        rewards = []
        successes = 0
        
        for _ in range(3):
            obs, _ = train_env.reset()
            ep_reward = 0
            
            for _ in range(100):
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = train_env.step(action)
                ep_reward += reward
                
                if terminated or truncated:
                    if info['success']:
                        successes += 1
                    break
            
            rewards.append(ep_reward)
        
        avg_reward = np.mean(rewards)
        success_rate = successes / 3
        
        print(f"   📈 Recompensa: {avg_reward:.2f}")
        print(f"   🎯 Éxito: {success_rate:.1%}")
    
    # Guardar modelo
    model.save("robot_arm_dimensions_fixed")
    print("💾 Modelo guardado como 'robot_arm_dimensions_fixed'")
    
    train_env.close()
    print("✅ Entrenamiento completado sin errores de dimensiones")
    
    return model

def demonstrate_fixed_robot():
    """Demostrar robot con dimensiones corregidas"""
    print("\n🎬 DEMOSTRACIÓN DEL ROBOT CORREGIDO")
    print("=" * 45)
    
    # Crear entorno visual
    demo_env = FixedRobotArmEnv(render_mode='human')
    
    # Cargar modelo
    try:
        model = PPO.load("robot_arm_dimensions_fixed")
        print("✅ Modelo cargado exitosamente")
    except:
        print("❌ Error: Modelo no encontrado. Ejecuta primero el entrenamiento.")
        demo_env.close()
        return
    
    print("🎯 Ejecutando episodios de demostración...")
    
    successful_episodes = 0
    
    for episode in range(5):
        print(f"\n--- 🎬 Episodio {episode + 1} ---")
        
        obs, _ = demo_env.reset()
        cup_pos = obs[6:9]  # Posición del vaso
        
        print(f"🎯 Vaso en: x={cup_pos[0]:.2f}, y={cup_pos[1]:.2f}")
        print("🤖 Brazo iniciando aproximación...")
        
        for step in range(demo_env.max_steps):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = demo_env.step(action)
            
            if step % 15 == 0:
                print(f"   Paso {step}: Distancia = {info['distance']:.3f}m")
            
            if terminated or truncated:
                if info['success']:
                    print("   🎉 ¡ÉXITO! Objetivo alcanzado")
                    successful_episodes += 1
                else:
                    print("   ⚠️ Intento parcial")
                
                # Pausa para observar resultado
                time.sleep(2)
                break
    
    print(f"\n📊 RESULTADOS FINALES:")
    print(f"🎯 Episodios exitosos: {successful_episodes}/5 ({successful_episodes/5:.1%})")
    
    demo_env.close()
    print("✅ Demostración completada sin errores")

In [4]:
# ============================================================================
# MENÚ PRINCIPAL CORREGIDO
# ============================================================================

def main_menu():
    """Menú principal con dimensiones corregidas"""
    while True:
        print("\n" + "="*60)
        print("🤖 BRAZO ROBÓTICO - DIMENSIONES CORREGIDAS")
        print("=" * 60)
        print("\n📋 OPCIONES:")
        print("1. 🧪 Probar dimensiones del entorno")
        print("2. 🚀 Entrenar agente (dimensiones corregidas)")
        print("3. 🎬 Demostrar robot entrenado")
        print("4. ⚡ Proceso completo (entrenamiento + demostración)")
        print("5. 🚪 Salir")
        
        choice = input("\n🎮 Selecciona (1-5): ").strip()
        
        if choice == "1":
            test_environment_dimensions()
        elif choice == "2":
            train_fixed_robot()
        elif choice == "3":
            demonstrate_fixed_robot()
        elif choice == "4":
            print("\n🔄 EJECUTANDO PROCESO COMPLETO...")
            confirm = input("¿Continuar con entrenamiento + demostración? (s/n): ").lower()
            if confirm in ['s', 'si', 'y', 'yes']:
                print("🚀 Iniciando entrenamiento...")
                train_fixed_robot()
                print("\n🎬 Iniciando demostración...")
                demonstrate_fixed_robot()
                print("\n🎉 ¡PROCESO COMPLETO FINALIZADO SIN ERRORES!")
            else:
                print("⏹️ Proceso cancelado")
        elif choice == "5":
            print("👋 ¡Hasta luego!")
            break
        else:
            print("❌ Opción inválida")


In [5]:
# ============================================================================
# EJECUCIÓN AUTOMÁTICA
# ============================================================================

if __name__ == "__main__":
    print("🎉 INICIANDO VERSIÓN CORREGIDA...")
    
    # Ejecutar prueba automática
    print("\n🔧 Ejecutando verificación automática...")
    try:
        test_environment_dimensions()
        print("✅ Verificación completada - Sin errores de dimensiones")
        
        # Mostrar menú
        main_menu()
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("💡 Reporta este error si persiste")

print("\n🏆 ¡CÓDIGO CORREGIDO Y LISTO PARA TU TAREA!")

🎉 INICIANDO VERSIÓN CORREGIDA...

🔧 Ejecutando verificación automática...

🧪 PROBANDO DIMENSIONES DEL ENTORNO
🤖 KUKA cargado con 7 articulaciones
🔗 Robot configurado con 7 articulaciones
🥤 Vaso creado en: (-0.31, -0.30, 0.06)
✅ Entorno creado - Observaciones: (13,)
✅ Acciones: (6,)
📊 Espacio de observaciones: (13,)
🎮 Espacio de acciones: (6,)
🥤 Vaso creado en: (-0.38, -0.39, 0.06)
🥤 Vaso creado en: (0.29, -0.34, 0.06)
🥤 Vaso creado en: (0.33, -0.21, 0.06)
✅ Entorno compatible con Stable-Baselines3

🔄 Probando reset...
🥤 Vaso creado en: (0.29, -0.00, 0.06)
✅ Reset OK - Observación: (13,)

▶️ Probando steps...
   Paso 0: obs=(13,), distancia=0.949
   Paso 10: obs=(13,), distancia=0.839
   Paso 20: obs=(13,), distancia=0.752
   Paso 30: obs=(13,), distancia=0.754
   Paso 40: obs=(13,), distancia=0.737
✅ Prueba de dimensiones completada exitosamente
✅ Verificación completada - Sin errores de dimensiones

🤖 BRAZO ROBÓTICO - DIMENSIONES CORREGIDAS

📋 OPCIONES:
1. 🧪 Probar dimensiones del ent

c:\Users\MSI LAPTOP\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


✅ PPO configurado correctamente
📚 Iniciando entrenamiento...

🌱 Exploración inicial - 15000 pasos
🥤 Vaso creado en: (-0.08, 0.08, 0.06)
🥤 Vaso creado en: (0.18, 0.32, 0.06)
🥤 Vaso creado en: (-0.40, 0.36, 0.06)
🥤 Vaso creado en: (-0.05, -0.17, 0.06)
🥤 Vaso creado en: (-0.13, 0.30, 0.06)
🥤 Vaso creado en: (-0.29, 0.24, 0.06)
🥤 Vaso creado en: (-0.39, -0.03, 0.06)
🥤 Vaso creado en: (-0.12, -0.17, 0.06)
🥤 Vaso creado en: (0.38, -0.04, 0.06)
🥤 Vaso creado en: (0.08, 0.07, 0.06)
🥤 Vaso creado en: (0.15, -0.33, 0.06)
🥤 Vaso creado en: (0.34, 0.26, 0.06)
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 171      |
|    ep_rew_mean     | 48.9     |
| time/              |          |
|    fps             | 674      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
🥤 Vaso creado en: (-0.19, -0.32, 0.06)
🥤 Vaso creado en: (-0.21, -0.29, 0.06)
🥤 Vaso creado en: (-0.33, 0.